In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_train = pd.read_csv("train.csv")
df_train.isna().sum()
df_train.info

id                   0
url_legal         2004
license           2004
excerpt              0
target               0
standard_error       0
dtype: int64

In [11]:
df_test = pd.read_csv("test.csv")
df_test.isna().sum()
df_test.info

<bound method DataFrame.info of           id                                    url_legal       license  \
0  c0f722661                                          NaN           NaN   
1  f0953f0a5                                          NaN           NaN   
2  0df072751                                          NaN           NaN   
3  04caf4e0c  https://en.wikipedia.org/wiki/Cell_division  CC BY-SA 3.0   
4  0e63f8bea      https://en.wikipedia.org/wiki/Debugging  CC BY-SA 3.0   
5  12537fe78                                          NaN           NaN   
6  965e592c0           https://www.africanstorybook.org/#     CC BY 4.0   

                                             excerpt  
0  My hope lay in Jack's promise that he would ke...  
1  Dotty continued to go to Mrs. Gray's every nig...  
2  It was a bright and cheerful scene that greete...  
3  Cell division is the process by which a parent...  
4  Debugging is the process of finding and resolv...  
5  To explain transitivity, let us lo